In [1]:
import os
os.listdir("/kaggle/input")

['titanic']

In [2]:
os.listdir("/kaggle/input/titanic")

['train.csv', 'test.csv', 'gender_submission.csv']

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load data
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)

train_data.head()

Train shape: (891, 12)
Test shape: (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].median())
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].median())

test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())

In [5]:
train_data['Sex'] = train_data['Sex'].map({'male': 0, 'female': 1})
test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1})

In [6]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
X = train_data[features]
y = train_data['Survived']
X_test = test_data[features]

In [7]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,0,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,3,1,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,3,0,35.0,0,0,8.0500


In [8]:
X.isnull().sum()

Pclass    0
Sex       0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64

In [9]:
# Split data into training and validation sets (80% train, 20% validate)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Initialize the model
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=1
)

# Train the model
model.fit(X_train, y_train)

# Make predictions on validation set
predictions = model.predict(X_val)

# Check accuracy
accuracy = accuracy_score(y_val, predictions)
print(f"Validation Accuracy: {accuracy:.2f}")

Validation Accuracy: 0.81


In [10]:
model.feature_importances_


array([0.13238025, 0.48774749, 0.11031612, 0.06003055, 0.03378282,
       0.17574276])

In [11]:
# Retrain on all training data
model.fit(X, y)

# Predict on the test data
test_predictions = model.predict(X_test)

# Create submission DataFrame
submission = pd.DataFrame({'PassengerId': test_data['PassengerId'],'Survived': test_predictions})

# Save to CSV
submission.to_csv('submission.csv', index=False)
print("Submission file saved as 'submision.csv'!")

Submission file saved as 'submision.csv'!
